# Greedy Algorithms

끝나는 시간이 가장 짧은 다음 activity를 고른다.

In [2]:
def printMaxActivities(s, f):
    n = len(f)
    i = 0
    for j in range(n):
        if s[j] >= f[i]:
            print(j)
            i = j

In [3]:
s = [1 , 3 , 0 , 5 , 8 , 5]
f = [2 , 4 , 6 , 7 , 9 , 9]
printMaxActivities(s , f)

1
3
4


In [5]:
# Time Complexity: O(nlogn)

def MaxActivities(arr, n):
    selected = []
    arr.sort(key = lambda x: x[1])
    i = 0
    selected.append(arr[i])
    for j in range(1, n):
        if arr[j][0] >= arr[i][1]:
            selected.append(arr[j])
            i = j
    return selected

In [7]:
Activity = [[5, 9], [1, 2], [3, 4], [0, 6],[5, 7], [8, 9]]
n = len(Activity)
selected = MaxActivities(Activity, n)
selected

[[1, 2], [3, 4], [5, 7], [8, 9]]

## Mininum Spanning Tree
Spanning Tree은 모든 vertices를 연결한 그래프이다.  
V=vertices의 갯수라고 하면 (V-1)개의 edges가 있다.

## Kruskal's Mininum Spanning Tree(MST)
1. 모든 edges를 오름차순으로 정렬한다.
2. 가장 작은 edge부터 cycle를 형성하는지 확인한다. cycle을 형성하지 않으면 포함시키고 아니면 버린다.

![](https://media.geeksforgeeks.org/wp-content/uploads/20210727035309/UntitledDiagram92.png)

아래와 같이 edge가 포함된다.
1. 7 -- 6 (1)
2. 6 -- 5 (2)
3. 8 -- 2 (2)
4. 0 -- 1 (4)
5. 2 -- 5 (4)
6. 2 -- 3 (7)
7. 0 -- 7 (8)
8. 3 -- 4 (9)

In [4]:
from collections import defaultdict

class Graph:

    def __init__(self, vertices):
        self.V = vertices
        self.graph = []

    def add_edge(self, u, v, w):
        self.graph.append([u, v, w])

    def find(self, parent, i): 
        if parent[i] == i:
            return i
        return self.find(parent, parent[i])

    def union(self, parent, rank, x, y):
        x_root = self.find(parent, x)
        y_root = self.find(parent, y)

        if rank[x_root] < rank[y_root]:
            parent[x_root] = y_root
        elif rank[x_root] > rank[y_root]:
            parent[y_root] = x_root
        else:
            parent[y_root] = x_root
            rank[x_root] += 1

    def KruskalMST(self):
        result = []
        i = 0
        e = 0
        self.graph = sorted(self.graph, key = lambda item:item[2]) # 거리 순으로 정렬한다.
        parent = []
        rank = []
        for node in range(self.V):
            parent.append(node) #[0,1,2,..]
            rank.append(0) #[0,0,0,..]

        while e < self.V - 1: 
            u, v, w = self.graph[i]
            i = i+1
            x = self.find(parent, u)
            y = self.find(parent, v)
            if x != y: # circle인지 확인한다. circle 형성이 안되면 포함시킨다.
                e = e+1
                result.append([u, v, w])
                self.union(parent, rank, x, y)
        
        minimum_cost = 0
        for u, v, weight in result:
            minimum_cost += weight
            print("%d -- %d == %d"%(u, v, weight))
        print("Minimum Spanning Tree : ", minimum_cost)

In [5]:
"""
            10
        0--------1
        | \      |
       6|   \5   |15
        |     \  |
        2--------3
             4     
"""
g = Graph(4)
g.add_edge(0, 1, 10)
g.add_edge(0, 2, 6)
g.add_edge(0, 3, 5)
g.add_edge(1, 3, 15)
g.add_edge(2, 3, 4)

g.KruskalMST()


2 -- 3 == 4
0 -- 3 == 5
0 -- 1 == 10
Minimum Spanning Tree :  19


## Prim's Minimum Spinning Tree
1. mstSet라는 집합을 생성한다.
2. 모든 값을 INFINITE로 초기화하여 첫 번째 vertex를 0부터 시작한다.
3. 모든 vertex를 포함할 때까지 다음을 반복한다
    - 포함된 vertex의 인접한 가장 짧은 edge를 가진 vertex를 추가한다.

![](https://www.geeksforgeeks.org/wp-content/uploads/Fig-11.jpg)

1. 0 -- 1 (4) {0,1}
2. 0 -- 7 (8) {0,1,7}
3. 7 -- 6 (1) {0,1,7,6}
4. 6 -- 5 (2) {0,1,7,6,5}
5. 2 -- 8 (2) {0,1,7,6,5,2,8}
6. 2 -- 3 (7) {0,1,7,6,5,2,8,3}
7. 3 -- 4 (9) {0,1,7,6,5,2,8,3,4}

In [14]:
import sys

class Graph:

    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0 for column in range(vertices) for row in range(vertices)]] 

    def printMST(self, parent):
        for i in range(1, self.V):
            print(parent[i], "-", i, "\t", self.graph[i][parent[i]])

    def minKey(self, key, mstSet):
        _min = sys.maxsize
        for v in range(self.V): # mstSet에 포함되지 않은 가장 작은 index를 찾는다.
            if key[v] < _min and mstSet[v] == False: 
                _min = key[v]
                min_index = v
        return min_index

    def primMST(self):
        key = [sys.maxsize] * self.V # 모든 vertices의 key값을 저장한다. 
        parent = [None] * self.V # parent node의 index를 저장한다. 
        key[0] = 0 # 첫번째 값은 0부터 시작
        mstSet = [False] * self.V # 포함 유무를 입력한다. (초기화는 전체 False)
        parent[0] = -1 

        for count in range(self.V):
            u = self.minKey(key, mstSet)
            mstSet[u] = True
            for v in range(self.V):
                if self.graph[u][v] > 0 and mstSet[v] == False and key[v] > self.graph[u][v]: # 연결된 vertex 중 포함되지 않고 가장 짧은 edge를 찾는다.
                    key[v] = self.graph[u][v]
                    parent[v] = u
        self.printMST(parent)


In [15]:
g = Graph(5)
g.graph = [ [0, 2, 0, 6, 0],
            [2, 0, 3, 8, 5],
            [0, 3, 0, 0, 7],
            [6, 8, 0, 0, 9],
            [0, 5, 7, 9, 0]]
g.primMST()

0 - 1 	 2
1 - 2 	 3
0 - 3 	 6
1 - 4 	 5


## Dijkstra's Shortest Path
1. 2가지 집합을 생성한다: 하나는 SPT에 포함된 vertices, 하나는 아직 포함되지 않은 집합이다.

## Huffman Coding